# End to end CNN for GTzan music classification EnvCNN


WINDOWED Version

Adapted by AL Koerich

To GTzan 10-fold

29 October 2018

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import soundfile as sf
from numpy import *
import soundfile as sf

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from keras import regularizers

import os, sys
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten
from keras.callbacks import TensorBoard
from keras.utils import np_utils, to_categorical
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
import keras.initializers as init

os.environ["CUDA_VISIBLE_DEVICES"]="2"
import tensorflow as tf
config = tf.ConfigProto( )
config.gpu_options.allow_growth = True
sess   = tf.Session(config=config)
import keras.backend.tensorflow_backend as tf_bkend
tf_bkend.set_session(sess)

Using TensorFlow backend.


In [3]:
#controling_Hyper parameters
batch_size = 100
nb_classes = 10
nb_epoch   = 300
frame_size = 110250

In [4]:
X_train = np.load( "/home-2/akoerich/GTzan_X_fold2-10_110250_75_256.npy" )
Y_train = np.load( "/home-2/akoerich/GTzan_Y_fold2-10_110250_75_256.npy" )
# t_train = np.load( "t_train_13_220500_50_256_box_id.npy" )
# s_train = np.load( "s_train_13_220500_50_256_box_id.npy" )


X_valid = np.load( "/home-2/akoerich/GTzan_X_fold1_110250_75_256.npy" )
Y_valid = np.load( "/home-2/akoerich/GTzan_Y_fold1_110250_75_256.npy" )
# t_valid = np.load( "t_valid_2_220500_50_256_box_id.npy" )
# s_valid = np.load( "s_valid_2_220500_50_256_box_id.npy" )

In [5]:
X_train.min(), X_train.max()

(-256.0, 256.0000000000001)

In [6]:
X_valid.min(), X_valid.max()

(-256.0, 256.0000000000001)

In [7]:
# If a validation split is not provided

#from sklearn.model_selection import train_test_split

#X_train, X_valid, Y_train, Y_valid = train_test_split(X_train2, Y_train2, test_size=0.10, stratify=Y_train2, random_state=42)

In [8]:
import gc
#X_train2 = None
#Y_train2 = None
#gc.collect()

In [9]:
X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape
#, t_train.shape, s_train.shape

((22490, 110250, 1), (22490, 10), (2500, 110250, 1), (2500, 10))

In [ ]:
def model_generator_soundnet():
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras.callbacks import ModelCheckpoint


    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=8, kernel_size=256, strides=2, activation='relu')(inp)
    norm1 = BatchNormalization()(conv1)
    pool1 = MaxPool1D(pool_size=8, strides=8)(norm1)
    #----------------------
    conv2 = Conv1D(filters=16, kernel_size=128, strides=2, activation='relu')(pool1)
    norm2 = BatchNormalization()(conv2)
    pool2 = MaxPool1D(pool_size=8, strides=8)(norm2)
    #----------------------
    conv3 = Conv1D(filters=32, kernel_size=64, strides=2, activation='relu')(pool2)
    norm3 = BatchNormalization()(conv3)
    #----------------------
    conv4 = Conv1D(filters=64, kernel_size=32, strides=2, activation='relu')(norm3)
    norm4 = BatchNormalization()(conv4)
    #----------------------
    conv5 = Conv1D(filters=128, kernel_size=16, strides=2, activation='relu')(norm4)
    norm5 = BatchNormalization()(conv5)
    pool3 = MaxPool1D(pool_size=4, strides=4)(norm5)

    flat = Flatten()(pool3)

    dense1 = Dense(1024, activation='relu')(flat)
    drop1  = Dropout(0.50)(dense1)
    
    # dense1 = Dense(2048, activation='relu')(flat)
    # drop1  = Dropout(0.50)(dense1)

    # dense2 = Dense(1024, activation='relu')(dense1)
    # drop2  = Dropout(0.50)(dense2)

    dense2a = Dense(512, activation='relu')(drop1)
    drop2a  = Dropout(0.50)(dense2a)
    
    dense3a = Dense(256, activation='relu')(drop2a)
    drop3a  = Dropout(0.50)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop3a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [ ]:
def model_generator_soundnet2():
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras.callbacks import ModelCheckpoint


    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=16, kernel_size=128, strides=2, activation='relu')(inp)
    norm1 = BatchNormalization()(conv1)
    pool1 = MaxPool1D(pool_size=8, strides=8)(norm1)
    #----------------------
    conv2 = Conv1D(filters=32, kernel_size=64, strides=2, activation='relu')(pool1)
    norm2 = BatchNormalization()(conv2)
    pool2 = MaxPool1D(pool_size=8, strides=8)(norm2)
    #----------------------
    conv3 = Conv1D(filters=64, kernel_size=32, strides=2, activation='relu')(pool2)
    norm3 = BatchNormalization()(conv3)
    #----------------------
    conv4 = Conv1D(filters=32, kernel_size=16, strides=2, activation='relu')(norm3)
    norm4 = BatchNormalization()(conv4)
    #----------------------
    #conv5 = Conv1D(filters=16, kernel_size=8, strides=2, activation='relu')(norm4)
    #norm5 = BatchNormalization()(conv5)
    #pool3 = MaxPool1D(pool_size=4, strides=4)(norm5)

    pool3 = MaxPool1D(pool_size=4, strides=4)(norm4)

    
    flat = Flatten()(pool3)

    dense1 = Dense(2048, activation='relu')(flat)
    drop1  = Dropout(0.50)(dense1)

    dense2 = Dense(1024, activation='relu')(drop1)
    drop2  = Dropout(0.50)(dense2)

    dense2a = Dense(512, activation='relu')(drop2)
    drop2a  = Dropout(0.50)(dense2a)
    
    dense3a = Dense(256, activation='relu')(drop2a)
    drop3a  = Dropout(0.50)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop3a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [ ]:
def model_generator_soundnet3():
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras.callbacks import ModelCheckpoint


    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=8, kernel_size=256, strides=2, activation='relu')(inp)
    norm1 = BatchNormalization()(conv1)
    pool1 = MaxPool1D(pool_size=8, strides=8)(norm1)
    #----------------------
    conv2 = Conv1D(filters=16, kernel_size=128, strides=2, activation='relu')(pool1)
    norm2 = BatchNormalization()(conv2)
    pool2 = MaxPool1D(pool_size=8, strides=8)(norm2)
    #----------------------
    conv3 = Conv1D(filters=32, kernel_size=64, strides=2, activation='relu')(pool2)
    norm3 = BatchNormalization()(conv3)
    #----------------------
    conv4 = Conv1D(filters=64, kernel_size=32, strides=2, activation='relu')(norm3)
    norm4 = BatchNormalization()(conv4)
    #----------------------
    conv5 = Conv1D(filters=128, kernel_size=16, strides=2, activation='relu')(norm4)
    norm5 = BatchNormalization()(conv5)
    pool3 = MaxPool1D(pool_size=4, strides=4)(norm5)

    flat = Flatten()(pool3)

    #dense1 = Dense(2048, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)

    dense2 = Dense(1024, activation='relu')(flat)
    drop2  = Dropout(0.50)(dense2)

    dense2a = Dense(512, activation='relu')(drop2)
    drop2a  = Dropout(0.50)(dense2a)
    
    dense3a = Dense(256, activation='relu')(drop2a)
    drop3a  = Dropout(0.50)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop3a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [ ]:
def model_generator_alenet():
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras.callbacks import ModelCheckpoint


    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=8, kernel_size=256, strides=2, activation='relu')(inp)
    norm1 = BatchNormalization()(conv1)
    # 8 -> 4
    pool1 = MaxPool1D(pool_size=4, strides=4)(norm1)
    #----------------------
    conv2 = Conv1D(filters=16, kernel_size=128, strides=2, activation='relu')(pool1)
    norm2 = BatchNormalization()(conv2)
    # 8 -> 4
    pool2 = MaxPool1D(pool_size=4, strides=4)(norm2)
    #----------------------
    conv3 = Conv1D(filters=32, kernel_size=64, strides=2, activation='relu')(pool2)
    norm3 = BatchNormalization()(conv3)
    #----------------------
    conv4 = Conv1D(filters=64, kernel_size=32, strides=2, activation='relu')(norm3)
    norm4 = BatchNormalization()(conv4)
    #----------------------
    conv5 = Conv1D(filters=128, kernel_size=16, strides=2, activation='relu')(norm4)
    norm5 = BatchNormalization()(conv5)
    pool3 = MaxPool1D(pool_size=2, strides=2)(norm5)

    flat = Flatten()(pool3)

    #dense1 = Dense(1024, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)
    
    #dense1 = Dense(2048, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)

    dense2 = Dense(1024, activation='relu')(flat)
    drop2  = Dropout(0.50)(dense2)

    dense2a = Dense(512, activation='relu')(drop2)
    drop2a  = Dropout(0.50)(dense2a)
    
    dense3a = Dense(256, activation='relu')(drop2a)
    drop3a  = Dropout(0.50)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop3a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [ ]:
# Best so far.
# Epoch 00300: val_acc did not improve from 0.50920

def model_generator_alenet2():
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras.callbacks import ModelCheckpoint


    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=16, kernel_size=128, strides=2, activation='relu')(inp)
    norm1 = BatchNormalization()(conv1)
    # 8 -> 4
    pool1 = MaxPool1D(pool_size=4, strides=4)(norm1)
    #----------------------
    conv2 = Conv1D(filters=8, kernel_size=256, strides=2, activation='relu')(pool1)
    norm2 = BatchNormalization()(conv2)
    # 8 -> 4
    pool2 = MaxPool1D(pool_size=4, strides=4)(norm2)
    #----------------------
    conv3 = Conv1D(filters=32, kernel_size=64, strides=2, activation='relu')(pool2)
    norm3 = BatchNormalization()(conv3)
    #----------------------
    conv4 = Conv1D(filters=64, kernel_size=32, strides=2, activation='relu')(norm3)
    norm4 = BatchNormalization()(conv4)
    #----------------------
    conv5 = Conv1D(filters=128, kernel_size=16, strides=2, activation='relu')(norm4)
    norm5 = BatchNormalization()(conv5)
    pool3 = MaxPool1D(pool_size=2, strides=2)(norm5)

    flat = Flatten()(pool3)

    #dense1 = Dense(1024, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)
    
    #dense1 = Dense(2048, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)

    dense2 = Dense(1024, activation='relu')(flat)
    drop2  = Dropout(0.50)(dense2)

    dense2a = Dense(512, activation='relu')(drop2)
    drop2a  = Dropout(0.50)(dense2a)
    
    dense3a = Dense(256, activation='relu')(drop2a)
    drop3a  = Dropout(0.50)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop3a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [ ]:
def model_generator_alenet3():
    from keras.layers import Input, Dense
    from keras.layers import AveragePooling1D
    from keras.models import Model
    from keras import initializers
    from keras.callbacks import ModelCheckpoint


    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=32, kernel_size=128, strides=2, activation='relu')(inp)
    norm1 = BatchNormalization()(conv1)
    # 8 -> 4
    pool1 = AveragePooling1D(pool_size=4, strides=4)(norm1)
    #----------------------
    conv2 = Conv1D(filters=16, kernel_size=256, strides=2, activation='relu')(pool1)
    norm2 = BatchNormalization()(conv2)
    # 8 -> 4
    pool2 = AveragePooling1D(pool_size=4, strides=4)(norm2)
    #----------------------
    conv3 = Conv1D(filters=32, kernel_size=64, strides=2, activation='relu')(pool2)
    norm3 = BatchNormalization()(conv3)
    #----------------------
    conv4 = Conv1D(filters=64, kernel_size=32, strides=2, activation='relu')(norm3)
    norm4 = BatchNormalization()(conv4)
    #----------------------
    conv5 = Conv1D(filters=128, kernel_size=16, strides=2, activation='relu')(norm4)
    norm5 = BatchNormalization()(conv5)
    pool3 = AveragePooling1D(pool_size=2, strides=2)(norm5)

    flat = Flatten()(pool3)

    #dense1 = Dense(1024, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)
    
    #dense1 = Dense(2048, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)

    dense2 = Dense(1024, activation='relu', kernel_initializer=initializers.glorot_uniform(seed=0) )(flat)
    drop2  = Dropout(0.55)(dense2)

    dense2a = Dense(512, activation='relu', kernel_initializer=initializers.glorot_uniform(seed=0) )(drop2)
    drop2a  = Dropout(0.50)(dense2a)
    
    dense3a = Dense(256, activation='relu', kernel_initializer=initializers.glorot_uniform(seed=0) )(drop2a)
    drop3a  = Dropout(0.50)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop3a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [ ]:
# best so far 0.53160
# dec 10. comecou dec 9 tarde.




def model_generator_alenet4():
    from keras.layers import Input, Dense
    from keras.layers import AveragePooling1D
    from keras.models import Model
    from keras import initializers
    from keras.callbacks import ModelCheckpoint
    from keras.layers import LeakyReLU    
    
    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=32, kernel_size=128, strides=2)(inp)
    norm1 = BatchNormalization()(conv1)
    act1  = LeakyReLU(alpha=0.1)(norm1)
    drop1 = Dropout(0.10)(act1)
    # 8 -> 4
    pool1 = AveragePooling1D(pool_size=4, strides=4)(drop1)
    #----------------------
    conv2 = Conv1D(filters=16, kernel_size=256, strides=2)(pool1)
    norm2 = BatchNormalization()(conv2)
    act2  = LeakyReLU(alpha=0.1)(norm2)
    drop2b = Dropout(0.10)(act2)
    # 8 -> 4
    pool2 = AveragePooling1D(pool_size=4, strides=4)(drop2b)
    #----------------------
    conv3 = Conv1D(filters=32, kernel_size=64, strides=2 )(pool2)
    norm3 = BatchNormalization()(conv3)
    act3  = LeakyReLU(alpha=0.1)(norm3)
    drop3b = Dropout(0.10)(act3)
    #----------------------
    conv4 = Conv1D(filters=64, kernel_size=32, strides=2 )(drop3b)
    norm4 = BatchNormalization()(conv4)
    act4  = LeakyReLU(alpha=0.1)(norm4)
    drop4b = Dropout(0.10)(act4)
    #----------------------
    conv5 = Conv1D(filters=128, kernel_size=16, strides=2 )(drop4b)
    norm5 = BatchNormalization()(conv5)
    act5  = LeakyReLU(alpha=0.1)(norm5)
    drop5b = Dropout(0.10)(act5)
    
    pool3 = AveragePooling1D(pool_size=2, strides=2)(drop5b)

    flat = Flatten()(pool3)

    #dense1 = Dense(1024, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)
    
    #dense1 = Dense(2048, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)

    dense2 = Dense(1024, activation='relu', kernel_initializer=initializers.glorot_uniform(seed=0) )(flat)
    drop2  = Dropout(0.55)(dense2)

    dense2a = Dense(512, activation='relu', kernel_initializer=initializers.glorot_uniform(seed=0) )(drop2)
    drop2a  = Dropout(0.55)(dense2a)
    
    dense3a = Dense(256, activation='relu', kernel_initializer=initializers.glorot_uniform(seed=0) )(drop2a)
    drop3a  = Dropout(0.50)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop3a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [13]:
def model_generator_alenet5():
    from keras.layers import Input, Dense
    from keras.layers import AveragePooling1D
    from keras.models import Model
    from keras import initializers
    from keras.callbacks import ModelCheckpoint
    from keras.layers import LeakyReLU    
    
    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=32, kernel_size=128, strides=2)(inp)
    norm1 = BatchNormalization()(conv1)
    act1  = LeakyReLU(alpha=0.1)(norm1)
    drop1 = Dropout(0.10)(act1)
    # 8 -> 4
    pool1 = AveragePooling1D(pool_size=4, strides=4)(drop1)
    #----------------------
    conv2 = Conv1D(filters=16, kernel_size=256, strides=2)(pool1)
    norm2 = BatchNormalization()(conv2)
    act2  = LeakyReLU(alpha=0.1)(norm2)
    drop2b = Dropout(0.10)(act2)
    # 8 -> 4
    pool2 = AveragePooling1D(pool_size=4, strides=4)(drop2b)
    #----------------------
    conv3 = Conv1D(filters=32, kernel_size=64, strides=2 )(pool2)
    norm3 = BatchNormalization()(conv3)
    act3  = LeakyReLU(alpha=0.1)(norm3)
    drop3b = Dropout(0.10)(act3)
    #----------------------
    conv4 = Conv1D(filters=64, kernel_size=32, strides=2 )(drop3b)
    norm4 = BatchNormalization()(conv4)
    act4  = LeakyReLU(alpha=0.1)(norm4)
    drop4b = Dropout(0.10)(act4)
    #----------------------
    conv5 = Conv1D(filters=128, kernel_size=16, strides=2 )(drop4b)
    norm5 = BatchNormalization()(conv5)
    act5  = LeakyReLU(alpha=0.1)(norm5)
    drop5b = Dropout(0.10)(act5)
    
    #pool3 = AveragePooling1D(pool_size=2, strides=2)(drop5b)

    flat = Flatten()(drop5b)

    #dense1 = Dense(1024, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)
    
    #dense1 = Dense(2048, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)

    dense2 = Dense(1024, kernel_initializer=initializers.glorot_uniform(seed=0) )(flat)
    actd2  = LeakyReLU(alpha=0.1)(dense2)
    drop2  = Dropout(0.6)(actd2)

    dense2a = Dense(512, kernel_initializer=initializers.glorot_uniform(seed=0) )(drop2)
    actd2a  = LeakyReLU(alpha=0.1)(dense2a)
    drop2a  = Dropout(0.5)(actd2a)
    
    #dense3a = Dense(256, activation='relu', kernel_initializer=initializers.glorot_uniform(seed=0) )(drop2a)
    #drop3a  = Dropout(0.50)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop2a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [14]:
fold = 2-10
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import time
hist = []


now = time.strftime("%c")
tbcallback = TensorBoard(log_dir='./tmp/GTzan_fold2-10_110250_75_256_'+now, histogram_freq=0, write_graph=True, write_images=True )

#for train_index, test_index in kf.split(X):
#generating the model 
model = model_generator_alenet5()

#checkpoints
str0="weights/"
str1="weights_GTzan_fold2-10_110250_75_256_pool_alenet5" 
str2=".best.hdf5" 
filepath=str0+str1+str2 
print(filepath)

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max') 
callbacks_list = [checkpoint, tbcallback]




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 110250, 1)         0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 55062, 32)         4128      
_________________________________________________________________
batch_normalization_6 (Batch (None, 55062, 32)         128       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 55062, 32)         0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 55062, 32)         0         
_________________________________________________________________
average_pooling1d_4 (Average (None, 13765, 32)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 6755, 16)          131088    
__________

In [15]:
#fitting the model 
hist.append(model.fit(X_train, Y_train,
                      batch_size = batch_size, 
                      epochs = nb_epoch,
                      verbose = 1,
                      shuffle = True,
                      callbacks = callbacks_list,
                      validation_data = (X_valid, Y_valid)
                     ))

Train on 22490 samples, validate on 2500 samples
Epoch 1/300
22490/22490 [==============================] - 198s 9ms/step - loss: 2.5996 - acc: 0.2498 - val_loss: 11.1404 - val_acc: 0.1092

Epoch 00001: val_acc improved from -inf to 0.10920, saving model to weights/weights_GTzan_fold2-10_110250_75_256_pool_alenet5.best.hdf5
Epoch 2/300
22490/22490 [==============================] - 197s 9ms/step - loss: 2.1846 - acc: 0.3458 - val_loss: 7.6018 - val_acc: 0.1060

Epoch 00002: val_acc did not improve from 0.10920
Epoch 3/300
22490/22490 [==============================] - 197s 9ms/step - loss: 2.0042 - acc: 0.4119 - val_loss: 6.5636 - val_acc: 0.1544

Epoch 00003: val_acc improved from 0.10920 to 0.15440, saving model to weights/weights_GTzan_fold2-10_110250_75_256_pool_alenet5.best.hdf5
Epoch 4/300
22490/22490 [==============================] - 197s 9ms/step - loss: 1.8253 - acc: 0.4667 - val_loss: 6.6294 - val_acc: 0.1780

Epoch 00004: val_acc improved from 0.15440 to 0.17800, saving mod


Epoch 00038: val_acc did not improve from 0.48960
Epoch 39/300
22490/22490 [==============================] - 198s 9ms/step - loss: 0.2735 - acc: 0.9115 - val_loss: 4.9620 - val_acc: 0.3204

Epoch 00039: val_acc did not improve from 0.48960
Epoch 40/300
22490/22490 [==============================] - 198s 9ms/step - loss: 0.2493 - acc: 0.9179 - val_loss: 3.4678 - val_acc: 0.4556

Epoch 00040: val_acc did not improve from 0.48960
Epoch 41/300
22490/22490 [==============================] - 197s 9ms/step - loss: 0.2563 - acc: 0.9173 - val_loss: 5.6617 - val_acc: 0.2904

Epoch 00041: val_acc did not improve from 0.48960
Epoch 42/300
22490/22490 [==============================] - 197s 9ms/step - loss: 0.2528 - acc: 0.9186 - val_loss: 5.9566 - val_acc: 0.3612

Epoch 00042: val_acc did not improve from 0.48960
Epoch 43/300
22490/22490 [==============================] - 197s 9ms/step - loss: 0.2428 - acc: 0.9226 - val_loss: 8.2518 - val_acc: 0.2468

Epoch 00043: val_acc did not improve from 0.


Epoch 00080: val_acc did not improve from 0.52880
Epoch 81/300
22490/22490 [==============================] - 199s 9ms/step - loss: 0.1590 - acc: 0.9485 - val_loss: 4.8271 - val_acc: 0.4772

Epoch 00081: val_acc did not improve from 0.52880
Epoch 82/300
22490/22490 [==============================] - 200s 9ms/step - loss: 0.1639 - acc: 0.9473 - val_loss: 5.7493 - val_acc: 0.4020

Epoch 00082: val_acc did not improve from 0.52880
Epoch 83/300
22490/22490 [==============================] - 199s 9ms/step - loss: 0.1596 - acc: 0.9488 - val_loss: 5.5534 - val_acc: 0.3940

Epoch 00083: val_acc did not improve from 0.52880
Epoch 84/300
22490/22490 [==============================] - 202s 9ms/step - loss: 0.1566 - acc: 0.9498 - val_loss: 4.0924 - val_acc: 0.4536

Epoch 00084: val_acc did not improve from 0.52880
Epoch 85/300
22490/22490 [==============================] - 200s 9ms/step - loss: 0.1601 - acc: 0.9487 - val_loss: 3.9229 - val_acc: 0.4420

Epoch 00085: val_acc did not improve from 0.

22490/22490 [==============================] - 199s 9ms/step - loss: 0.1399 - acc: 0.9550 - val_loss: 3.2853 - val_acc: 0.4732

Epoch 00123: val_acc did not improve from 0.52880
Epoch 124/300
22490/22490 [==============================] - 199s 9ms/step - loss: 0.1437 - acc: 0.9545 - val_loss: 3.3701 - val_acc: 0.4132

Epoch 00124: val_acc did not improve from 0.52880
Epoch 125/300
22490/22490 [==============================] - 199s 9ms/step - loss: 0.1374 - acc: 0.9557 - val_loss: 3.8740 - val_acc: 0.4580

Epoch 00125: val_acc did not improve from 0.52880
Epoch 126/300
22490/22490 [==============================] - 199s 9ms/step - loss: 0.1318 - acc: 0.9568 - val_loss: 4.0584 - val_acc: 0.4308

Epoch 00126: val_acc did not improve from 0.52880
Epoch 127/300
22490/22490 [==============================] - 199s 9ms/step - loss: 0.1396 - acc: 0.9544 - val_loss: 5.8172 - val_acc: 0.4508

Epoch 00127: val_acc did not improve from 0.52880
Epoch 128/300
22490/22490 [===========================

22490/22490 [==============================] - 202s 9ms/step - loss: 0.1315 - acc: 0.9574 - val_loss: 6.4520 - val_acc: 0.4252

Epoch 00166: val_acc did not improve from 0.52880
Epoch 167/300
22490/22490 [==============================] - 202s 9ms/step - loss: 0.1210 - acc: 0.9598 - val_loss: 3.9489 - val_acc: 0.4540

Epoch 00167: val_acc did not improve from 0.52880
Epoch 168/300
22490/22490 [==============================] - 202s 9ms/step - loss: 0.1253 - acc: 0.9578 - val_loss: 3.9225 - val_acc: 0.4692

Epoch 00168: val_acc did not improve from 0.52880
Epoch 169/300
22490/22490 [==============================] - 202s 9ms/step - loss: 0.1284 - acc: 0.9577 - val_loss: 3.0798 - val_acc: 0.4624

Epoch 00169: val_acc did not improve from 0.52880
Epoch 170/300
22490/22490 [==============================] - 202s 9ms/step - loss: 0.1301 - acc: 0.9578 - val_loss: 5.2378 - val_acc: 0.3764

Epoch 00170: val_acc did not improve from 0.52880
Epoch 171/300
22490/22490 [===========================


Epoch 00208: val_acc did not improve from 0.52880
Epoch 209/300
22490/22490 [==============================] - 360s 16ms/step - loss: 0.1177 - acc: 0.9610 - val_loss: 4.6036 - val_acc: 0.3980

Epoch 00209: val_acc did not improve from 0.52880
Epoch 210/300
22490/22490 [==============================] - 368s 16ms/step - loss: 0.1196 - acc: 0.9607 - val_loss: 3.8372 - val_acc: 0.4788

Epoch 00210: val_acc did not improve from 0.52880
Epoch 211/300
22490/22490 [==============================] - 365s 16ms/step - loss: 0.1188 - acc: 0.9602 - val_loss: 4.7969 - val_acc: 0.4464

Epoch 00211: val_acc did not improve from 0.52880
Epoch 212/300
22490/22490 [==============================] - 371s 17ms/step - loss: 0.1194 - acc: 0.9602 - val_loss: 4.4915 - val_acc: 0.4568

Epoch 00212: val_acc did not improve from 0.52880
Epoch 213/300
22490/22490 [==============================] - 370s 16ms/step - loss: 0.1209 - acc: 0.9598 - val_loss: 4.9544 - val_acc: 0.3968

Epoch 00213: val_acc did not impro

22490/22490 [==============================] - 285s 13ms/step - loss: 0.1127 - acc: 0.9609 - val_loss: 3.0387 - val_acc: 0.4820

Epoch 00251: val_acc did not improve from 0.52880
Epoch 252/300
22490/22490 [==============================] - 287s 13ms/step - loss: 0.1161 - acc: 0.9608 - val_loss: 3.9678 - val_acc: 0.4828

Epoch 00252: val_acc did not improve from 0.52880
Epoch 253/300
22490/22490 [==============================] - 279s 12ms/step - loss: 0.1130 - acc: 0.9611 - val_loss: 5.0687 - val_acc: 0.4056

Epoch 00253: val_acc did not improve from 0.52880
Epoch 254/300
22490/22490 [==============================] - 290s 13ms/step - loss: 0.1104 - acc: 0.9613 - val_loss: 4.1205 - val_acc: 0.4572

Epoch 00254: val_acc did not improve from 0.52880
Epoch 255/300
22490/22490 [==============================] - 310s 14ms/step - loss: 0.1119 - acc: 0.9622 - val_loss: 4.1355 - val_acc: 0.4716

Epoch 00255: val_acc did not improve from 0.52880
Epoch 256/300
22490/22490 [======================


Epoch 00293: val_acc did not improve from 0.52880
Epoch 294/300
22490/22490 [==============================] - 300s 13ms/step - loss: 0.1075 - acc: 0.9625 - val_loss: 4.7177 - val_acc: 0.4628

Epoch 00294: val_acc did not improve from 0.52880
Epoch 295/300
22490/22490 [==============================] - 316s 14ms/step - loss: 0.1142 - acc: 0.9610 - val_loss: 3.8063 - val_acc: 0.4856

Epoch 00295: val_acc did not improve from 0.52880
Epoch 296/300
22490/22490 [==============================] - 313s 14ms/step - loss: 0.1078 - acc: 0.9627 - val_loss: 4.3796 - val_acc: 0.4172

Epoch 00296: val_acc did not improve from 0.52880
Epoch 297/300
22490/22490 [==============================] - 287s 13ms/step - loss: 0.1068 - acc: 0.9640 - val_loss: 4.1973 - val_acc: 0.4936

Epoch 00297: val_acc did not improve from 0.52880
Epoch 298/300
22490/22490 [==============================] - 288s 13ms/step - loss: 0.1099 - acc: 0.9626 - val_loss: 3.7257 - val_acc: 0.4868

Epoch 00298: val_acc did not impro

In [16]:
filepath

'weights/weights_GTzan_fold2-10_110250_75_256_pool_alenet5.best.hdf5'

https://www.music-ir.org/nema_out/mirex2017/results/act/latin_report/accperfold.html

MIREX 2017:



Summary Results    [top]
Algorithm
Classification Accuracy
Normalised Classification Accuracy
LPNKK1
0.7586
0.7571
PLNPH1
0.6619
0.6573
XLJ1
0.6148
0.6079
LPNKK3
0.7347
0.7324
LPNKK2
0.6511
0.6458





Fold
LPNKK1
LPNKK2
LPNKK3
PLNPH1
XLJ1
1
0.7186
0.6041
0.6860
0.5795
0.5453
0
0.7627
0.6426
0.7444
0.6772
0.5998
2
0.8055
0.7194
0.7872
0.7518
0.7183


/Users/akoerich/Dropbox/Mendeley/pdf/Aytar, Vondrick, Torralba - 2016.pdf

/Users/akoerich/HOME/ETS/Etudiants/Sajjad Abdoli/DGA1032/DGA1032-Dissertation_abdoli.pdf